#  FABRIC Storage: Reserve and Benchmark

FABRIC has three types of reservable storage:
 - Local Disk (VM)
 - NVMe (dedicated PCI device)
 - Network Storage -- Coming Soon

This example notebook will demonstrate how to reserve and use each type of storage. The example will also benchmark each type of storage in order to show its performance.

## Configure the Environment

In [1]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

In [8]:
ssh_key_file_priv=os.environ['HOME']+"/.ssh/id_rsa"
ssh_key_file_pub=os.environ['HOME']+"/.ssh/id_rsa.pub"

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

In [9]:
credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
print(f"FABRIC Credential Manager   : {credmgr_host}")

orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"FABRIC Orchestrator         : {orchestrator_host}")

FABRIC Credential Manager   : cm.fabric-testbed.net
FABRIC Orchestrator         : orchestrator.fabric-testbed.net


## Create Slice Manager Object
Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters in the refresh call below.

In [10]:
slice_manager = SliceManager(oc_host=orchestrator_host, 
                             cm_host=credmgr_host ,
                             project_name='all', 
                             scope='all')

# Initialize the slice manager
slice_manager.initialize()

## Create a Node

The cell below creates a slice that contains a single node with a 1TB NVMe device. 


### Set the Slice Name and FABRIC Site

In [5]:
from fabrictestbed.slice_editor import ComponentType

slice_name="MySlice2"
site_name="STAR"
node_name='Node1'
image = 'default_centos_8'
image_type = 'qcow2'
cores = 4
ram = 32
disk = 1000

nvme_component_type = ComponentType.NVME
nvme_model = 'P4510'
nvme_name = 'nvme1'

In [6]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities 
# Create topology
t = ExperimentTopology()

# Add node
n1 = t.add_node(name=node_name, site=site_name)

# Set capacities
cap = Capacities()
cap.set_fields(core=cores, ram=ram, disk=disk)

# Set Properties
n1.set_properties(capacities=cap, image_type=image_type, image_ref=image)

# Add the PCI NVMe device
n1.add_component(ctype=nvme_component_type, model=nvme_model, name=nvme_name)

# Generate Slice Graph
slice_graph = t.serialize()

# Request slice from Orchestrator
return_status, slice_reservations = slice_manager.create(slice_name=slice_name, 
                                            slice_graph=slice_graph, 
                                            ssh_key=ssh_key_pub)

if return_status == Status.OK:
    slice_id = slice_reservations[0].get_slice_id()
    print("Submitted slice creation request. Slice ID: {}".format(slice_id))
else:
    print(f"Failure: {slice_reservations}")

Submitted slice creation request. Slice ID: a1b6bc4b-85b6-4681-a6e5-e21b5353a380


## Get the Slice

In [7]:
import time
def wait_for_slice(slice,timeout=180,interval=10,progress=False):
    timeout_start = time.time()

    if progress: print("Waiting for slice .", end = '')
    while time.time() < timeout_start + timeout:
        return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

        if return_status == Status.OK:
            slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
            if slice.slice_state == "StableOK":
                if progress: print(" Slice state: {}".format(slice.slice_state))
                return slice
            if slice.slice_state == "Closing" or slice.slice_state == "Dead":
                if progress: print(" Slice state: {}".format(slice.slice_state))
                return slice    
        else:
            print(f"Failure: {slices}")
        
        if progress: print(".", end = '')
        time.sleep(interval)
    
    if time.time() >= timeout_start + timeout:
        if progress: print(" Timeout exceeded ({} sec). Slice: {} ({})".format(timeout,slice.slice_name,slice.slice_state))
        return slice    


return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

if return_status == Status.OK:
    slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
    slice = wait_for_slice(slice, progress=True)

print()
print("Slice Name : {}".format(slice.slice_name))
print("ID         : {}".format(slice.slice_id))
print("State      : {}".format(slice.slice_state))
print("Lease End  : {}".format(slice.lease_end))

Waiting for slice ....

IndexError: list index out of range

## Get the Node

Retrieve the node information and save the management IP address.


In [ ]:
return_status, experiment_topology = slice_manager.get_slice_topology(slice_object=slice)

node = experiment_topology.nodes[node_name]

management_ip = str(node.get_property(pname='management_ip'))
print("Node Name        : {}".format(node.name))
print("Management IP    : {}".format(management_ip))
print()

nvme1 = node.components[nvme_name]
print("NVMe Name        : {}".format(nvme1.name))
print("Details          : {}".format(nvme1.details))
print("Disk (G)         : {}".format(nvme1.get_property(pname='capacity_allocations').disk))
print("Units            : {}".format(nvme1.get_property(pname='capacity_allocations').unit))
print("PCI Address      : {}".format(nvme1.get_property(pname='label_allocations').bdf))
print("Model            : {}".format(nvme1.model))
print("Type             : {}".format(nvme1.get_property(pname='type')))


## Setup SSH Connection for Commands

Setup <code>paramiko</code> to send commands to the node using <code>ssh</code>.

In [ ]:
import paramiko

key = paramiko.RSAKey.from_private_key_file(ssh_key_file_priv)
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())

client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(management_ip,username='centos',pkey = key)

## Configure the Node's Storage

### Local Disk

Run the command <code>df -h /</code> on the node. You should see a 1 TB root file system on the the local disk. This is the local disk used by the VM and is mounted at <code>/</code>.  

There is no need to configure the local disk.

In [ ]:
stdin, stdout, stderr = client.exec_command('df -h /')

print (str(stdout.read(),'utf-8').replace('\\n','\n'))

### NVMe PCI Device

Run the command <code>lspci</code> to see your NVMe PCI device. This is the raw NVMe PCI device that is not yet formated or mounted.

In [ ]:
stdin, stdout, stderr = client.exec_command('lspci | grep NVMe')

print (str(stdout.read(),'utf-8').replace('\\n','\n'))

Run <code>fdisk</code> to see the NVMe block device. This device is not yet formated. We will partition and format this device later.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo fdisk -l /dev/nvme*')

print (str(stdout.read(),'utf-8').replace('\\n','\n'))

### Network Attached Storage

Network attached storage is not yet available on FABRIC. This notebook will be updated to when it becomes available.

## Configure and Benchmark the Storage Devices

We will use <code>dd</code> to perform a simple benchmark of the different storage devices.  

Note that this is not a complete evaluation of  FABRIC storage devices and is meant more as a exercise for learning about using FABRIC storage and its performance.

### Local Disk

As seen above the local disk (`/dev/vda1`) is mounted at `/`.  We can read/write to the `/tmp` directory to benchmark this disk.

We can verify that `/tmp` is part of `/dev/vda1` by issuing the command below.

In [ ]:
stdin, stdout, stderr = client.exec_command('df /tmp')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Now lets try writing a modest 1G file to the local disk using a simple `dd` command. We will use a 1G block size to simulate a full 1G file being written. If you would like to simulate a lot of smaller files you should reduce the value of `bs` to the size of the files and increase `count` to the number of files.

In [ ]:
command='dd if=/dev/zero of=/tmp/output bs=1G count=1'
stdin, stdout, stderr = client.exec_command(command)
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

The 1G file was probably written at over 1 GB/s. This is much faster than the local disk. How did this happen? The answer is that there the OS and VM hypervisor both have buffers in memory that allow bursts of file writes to be quickly written memory and, later, transfered to disk. This optimization helps many applications but is limited by the memory available for disk caching. 

Typically, storage benchmarking aims to test performance of the stoarge, not the OS buffering system. Some `dd` options can help with this.  Let's try again but add the `oflag=direct` option that will skip the file system buffer used by the VM's OS.

In [ ]:
command='dd if=/dev/zero of=/tmp/output bs=1G count=1 oflag=direct'
stdin, stdout, stderr = client.exec_command(command)
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

What did you find? Often, this produces a counterintuitive result. It is not surprising to see speeds over 2.5 GB/s.  What is happening here?  

The increased performance is due to the hypervisor's virtual block device cache. Although this cache was also used in the previous step, it seems that it can enable higher bandwidth on its own than when used in combination with the VM's file system cache. 

Skipping this cache and writing directly to the physical block device is not possible without re-configuring the hypervisor. You will not be able to perform this test on FABRIC. You can, however, write a large enough file that the cache fills early in the write and the amortized performance approaches the block devices' write performance.

Try writing larger files. Note these tests could take tens of minutes so be patient.

Write a 10G file:

In [ ]:
command='dd if=/dev/zero of=/tmp/output bs=5G count=1 oflag=direct'
stdin, stdout, stderr = client.exec_command(command)
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Write a 20G file:

In [ ]:
command='dd if=/dev/zero of=/tmp/output bs=25G count=1 oflag=direct'
stdin, stdout, stderr = client.exec_command(command)
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

The maximum value of `bs` is limited by the size of the VM's memory. Very large write tests must have reasonable values of `bs` and increased `count`. 

Try larger write tests. These tests may take some time. Be patient. 

In [ ]:
stdin, stdout, stderr = client.exec_command('dd if=/dev/zero of=/tmp/output bs=25G count=16 oflag=direct')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

If you tests were large enough you will approach the write bandwidth of the disk. In our tests, this should be a bit under 200 MB/s.

### NVMe storage.

NVMe storage is provided as bare PCI block devices and will likely need to be partitioned and formated before use.

Find the NVMe block device.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo fdisk -l /dev/nvme*')

print (str(stdout.read(),'utf-8').replace('\\n','\n'))

There are no partitions or files systems on this device. First, let's partition the disk.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo parted /dev/nvme0n1 mklabel gpt')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

We can check that it has been created using the command below.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo parted /dev/nvme0n1 print')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Next, let's print the free space in the NVMe drive in megabytes.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo parted /dev/nvme0n1 print unit MB print free')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

We can see that the disk is 1GB. Next, let's create a partition that uses up this entire space.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo parted --align optimal /dev/nvme0n1 mkpart primary ext4 0% 1000205MB')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Let's examine the partitions one more time to see the partition we created.

In [ ]:
stdin, stdout, stderr = client.exec_command('lsblk /dev/nvme0n1')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Now, let's format the disk with a `ext4` file system.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo mkfs.ext4 /dev/nvme0n1p1')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Next, we need to mount the file system. 

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo mkdir /mnt/nvme_mount && sudo mount /dev/nvme0n1p1 /mnt/nvme_mount')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Confirm that the file system is mounted and the correct size.

In [ ]:
stdin, stdout, stderr = client.exec_command('df -h /mnt/nvme_mount')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Now we can use `dd` command to benchmark the NVMe drive just like we did the local disk. Unlike the local disk, our VMs have direct control of the NVMe PCI devices so there is no hypervisor cache. Using `oflag=direct` will get closet to the actual performance of the NVMe block device.
 
Try a 1G file without `oflag=direct`:

In [ ]:
command='sudo dd if=/dev/zero of=/mnt/nvme_mount/output bs=1G count=1'
stdin, stdout, stderr = client.exec_command(command)
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Try a 1G file with `oflag=direct`:

In [ ]:
command='sudo dd if=/dev/zero of=/mnt/nvme_mount/output bs=1G count=1 oflag=direct'
stdin, stdout, stderr = client.exec_command(command)
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Try a 5G file with `oflag=direct`:

In [ ]:
command='sudo dd if=/dev/zero of=/mnt/nvme_mount/output bs=5G count=1 oflag=direct'
stdin, stdout, stderr = client.exec_command(command)
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Try a larger files with `oflag=direct`:

In [ ]:
command='sudo dd if=/dev/zero of=/mnt/nvme_mount/output bs=1G count=100 oflag=direct'
stdin, stdout, stderr = client.exec_command(command)
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

You should be seeing much higher bandwidths than with the local disk.

## Cleanup Your Experiment

In [ ]:
status, result = slice_manager.delete(slice_id=slice_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))